# Projeto 1 - Ciência dos Dados

Nome: Layne Silva

Nome: Lidia Domingos

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [41]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [42]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\layneps\Desktop\Insper\CDados\PROJETO 1\PROJETO-1-CDADOS


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [43]:
filename = 'XIAOMI.xlsx'

In [44]:
train = pd.read_excel(filename)
train.head(5)

,TREINAMENTO,CLASSIFICAÇÃO
0,quem aqui tb pronuncia xiaomi xaiomi,0
1,xiaomi mi true airdots 2s earphone(aliexpress)...,0
2,@felpsflip carreguei meu xiaomi uma vez na vid...,1
3,"#promo #\n$133.99 ~ 111,85 € ✂️cupão/cupom/cup...",0
4,"#promo #\n$299.99 ~ 250,41 € ✂️cupão/cupom/cup...",0


In [45]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,CLASSIFICAÇÃO
0,"#promo #\n$11.99 ~ 10,01 € ✂️cupão/cupom/cupon...",0
1,"💠xiaomi poco m3 \n\n💵 por: r$ 728,27 64gb\n💵 ...",0
2,"@_faccin_ ""quem ai tem um xiaomi pra ligar o p...",0
3,comprei mais capinha pro xiaomi https://t.co/7...,0
4,#promoção #oferta \n\n💵 por: r$ 1420 \n🛒 link:...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

*Escolhida a empresa XIAOMI, classificamos como tweets relevantes toda e qualquer citação (seja ela positiva ou negativa) dos internautas falando sobre o produto. Tweets considerados como irrelevantes seriam tweets não relacionados diretamente ao produto.*

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [46]:
#Limpeza de mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:"?;\n)@("/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
    

In [52]:
#JUNTANDO TODAS PALAVRAS DOS TWEETS DO TREINAMENTO
join_train_global = ''.join(train.TREINAMENTO) 

#JUNTANDO TODAS AS PALAVRAS DOS TWEETS RELEVANTES DO TREINAMENTO
train_relevante = train.loc[train["CLASSIFICAÇÃO"] == 1, :]
join_train_relevante = ''.join(train_relevante.TREINAMENTO) 

#JUNTANDO TODAS AS PALAVRAS DOS TWEETS IRRELEVANTES DO TREINAMENTO
train_irrelevante = train.loc[train["CLASSIFICAÇÃO"] == 0, :]
join_train_irrelevante = ''.join(train_irrelevante.TREINAMENTO) 


In [60]:
#LIMPANDO VARIÁVEIS CRIADAS, COLOCANDO EM MINÚSCULO E AS GUARDANDO
train_global_limpas = cleanup(join_train_global.lower())
train_relevante_limpas = cleanup(join_train_relevante.lower())
train_irrelevante_limpas = cleanup(join_train_irrelevante.lower())


In [63]:
#QUEBRANDO EM PALAVRAS

quebra_train_global = join_train_global.split()
quebra_train_relevante = join_train_relevante.split()
quebra_train_irrelevante = join_train_irrelevante.split()

#TRANSFORMANDO EM PD.SERIES
series_train_global = pd.Series(quebra_train_global)
series_train_relevante = pd.Series(quebra_train_relevante)
series_train_irrelevante = pd.Series(quebra_train_irrelevante)

In [ ]:
#Correção de espaços entre palavras e/ou emojis.


In [ ]:
#Proposta de outras limpezas/transformações que não afetem a qualidade da informação.

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**